In [5]:
import scanpy as sc
import numpy as np

In [2]:

adata = sc.read_h5ad("/lustre/groups/ml01/workspace/ot_perturbation/data/zebrafish_new/zebrafish_processed.h5ad")

In [14]:
ct_oligos = []
for oligo in adata.obs["Oligo"].unique():
    adata_oligo = adata[adata.obs["Oligo"]==oligo]
    tp = adata_oligo.obs["timepoint"].unique()[0]
    gene_target = adata_oligo.obs["gene_target"].unique()[0]
    ct_oligo = adata_oligo.obs[f"cell_type_broad"].value_counts().to_frame()
    ct_oligo/=ct_oligo.sum()

    adata_ctrl = adata[(adata.obs["timepoint"]==tp) & (adata.obs["gene_target"]=="control_control")]
    ct_ctrl = adata_ctrl.obs[f"cell_type_broad"].value_counts().to_frame()
    ct_ctrl/=ct_ctrl.sum()
    ct_oligo["ctrl_count"] = ct_ctrl["count"]
    ct_oligo["ratio"] = ct_oligo["count"]/ct_oligo["ctrl_count"]
    ct_oligo["logratio"] = np.log2(ct_oligo["ratio"])
    ct_oligo["Oligo"] = oligo
    ct_oligo["timepoint"] = tp
    ct_oligo["condition"] = adata_oligo.obs["condition"].unique()[0]
    ct_oligos.append(ct_oligo)

In [16]:
import pandas as pd
all_ratios = pd.concat(ct_oligos)

In [17]:
all_ratios.shape

(63182, 7)

In [19]:
all_ratios["gene_target"] = all_ratios.apply(lambda x: "_".join(x["condition"].split("_")[:2]), axis=1)

In [20]:
all_ratios[all_ratios["gene_target"]=="control_control"]

,count,ctrl_count,ratio,logratio,Oligo,timepoint,condition,gene_target
cell_type_broad,,,,,,,,
neural progenitor (hindbrain),0.106569,0.091422,1.165683,0.221176,18h_ctrl-inj_P13_H1,18.0,control_control_18,control_control
"eye, optic cup",0.058394,0.049256,1.185526,0.245528,18h_ctrl-inj_P13_H1,18.0,control_control_18,control_control
neural progenitor (telencephalon/diencephalon),0.047689,0.045076,1.057951,0.081272,18h_ctrl-inj_P13_H1,18.0,control_control_18,control_control
connective tissue-meninges-dermal FB,0.045742,0.053299,0.858222,-0.220578,18h_ctrl-inj_P13_H1,18.0,control_control_18,control_control
fast-committed myocyte (pre-fusion),0.044769,0.025487,1.756517,0.812718,18h_ctrl-inj_P13_H1,18.0,control_control_18,control_control
...,...,...,...,...,...,...,...,...
olfactory vesicle/bulb,0.000829,0.003418,0.242405,-2.044509,18h_ctrl-mafba_P9_H1,18.0,control_control_18,control_control
dorsal spinal cord neuron,0.000829,0.000176,4.713431,2.236778,18h_ctrl-mafba_P9_H1,18.0,control_control_18,control_control
pancreas,0.000829,0.000508,1.631572,0.706263,18h_ctrl-mafba_P9_H1,18.0,control_control_18,control_control


In [21]:
all_ratios.to_csv("/lustre/groups/ml01/workspace/cell_flow_zebrafish/results/single_condition/cell_type_proportion_baseline/all_oligos.csv")

In [26]:
ct_oligo_ratios = []
for oligo in adata.obs["Oligo"].unique():
    adata_oligo = adata[adata.obs["Oligo"]==oligo]
    tp = adata_oligo.obs["timepoint"].unique()[0]
    ood_cond = adata_oligo.obs["condition"].unique()[0]
    gene_target = adata_oligo.obs["gene_target"].unique()[0]
    ct_oligo = adata_oligo.obs[f"cell_type_broad"].value_counts().to_frame()
    ct_oligo/=ct_oligo.sum()

    adata_true = adata[adata.obs["condition"]==ood_cond]
    ct_true = adata_true.obs[f"cell_type_broad"].value_counts().to_frame()
    ct_true/=ct_true.sum()


    adata_ctrl = adata[(adata.obs["timepoint"]==tp) & (adata.obs["gene_target"]=="control_control")]
    ct_ctrl = adata_ctrl.obs[f"cell_type_broad"].value_counts().to_frame()
    ct_ctrl/=ct_ctrl.sum()

    ct_oligo["ctrl"] = ct_ctrl["count"]
    ct_true["ctrl"] = ct_ctrl["count"]

    ct_oligo["ratio_oligo"] = ct_oligo["count"]/ct_oligo["ctrl"]
    ct_true["ratio_true"] = ct_true["count"]/ct_true["ctrl"]
    ct_oligo["ratio_true"] = ct_true["ratio_true"]

    ct_oligo = ct_oligo[["ratio_oligo", "ratio_true"]]
    ct_oligo["logratio_true"] = np.log2(ct_oligo["ratio_true"])
    ct_oligo["logratio_oligo"] = np.log2(ct_oligo["ratio_oligo"])

    ct_oligo["Oligo"] = oligo
    ct_oligo["timepoint"] = tp
    ct_oligo["condition"] = ood_cond
    ct_oligo["gene_target"] = gene_target
    ct_oligo_ratios.append(ct_oligo)

In [29]:
all_ratios = pd.concat(ct_oligo_ratios)

In [30]:
all_ratios.to_csv("/lustre/groups/ml01/workspace/cell_flow_zebrafish/results/single_condition/cell_type_proportion_baseline/all_single_oligo_ratios.csv")
